# Saliency maps
This notebook will walk you through computation of a saliency map for an image or a frame of a video. We will use this repository - which you will need to clone into your ~/Code folder! 

To do this, please call the following at the command line (terminal on phi):
```
$ cd ~/Code/
$ git clone https://github.com/akisatok/pySaliencyMap.git
```



In [ ]:
import file_io
import plot_utils
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

from IPython.display import Image

# Define the Session loader
%run ../../code/vedb_utils.py

In [ ]:
# This command will run the main useful script in the repo, 
# which defines some useful functions we will use below.
%run ~/Code/pySaliencyMap/pySaliencyMap.py

First, we will compute the saliency of some of their demo images. The functions in this library are based on opencv, and thus want BGR image inputs, so we will use opencv to read in an image instead of matplotlib. 

In [ ]:
file_path = '/home/mark/Code/pySaliencyMap/test2.png'
image = cv2.imread(file_path)

The way this repository works, a python *class* is created, which knows about the size of the images or frames that you want to process, and a method of that object actually computes the saliecy of input frames. 

In [ ]:
# Create a saliency map generator
image_shape = image.shape[:2]
saliency_generator = pySaliencyMap(*image_shape)

Compute saliency!

In [ ]:
# "SM" here is "Saliency Map" - so, Saliency Map -> get Saliency Map!
saliency_map = saliency_generator.SMGetSM(image)

Show the map! The highest values are for the bar that is not the same color as the rest.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
# For display, convert image back to RGB by reversing last axis
_ = ax[0].imshow(image[:, :, ::-1])
im_handle = ax[1].imshow(saliency_map, cmap='inferno')
plt.colorbar(im_handle)

Same thing, for a different image. It's left as an exercise to you to compute saliency maps for the other two example images, if you wish, or to explore the rest of this repo (again, it's here: https://github.com/akisatok/pySaliencyMap)

Below, we compute a saliency map for frames of VEDB data.

In [ ]:
file_path = '/home/mark/Code/pySaliencyMap/test3.jpg'
# The functions in this library want BGR image inputs, so use opencv 
# to read in image
image = cv2.imread(file_path)
# Create a saliency map generator
image_shape = image.shape[:2]
saliency_generator = pySaliencyMap(*image_shape)
# "SM" here is "Saliency Map" - so, Saliency Map -> get Saliency Map!
saliency_map = saliency_generator.SMGetSM(image)
# Show map
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
# For display, convert image back to RGB by reversing last axis
_ = ax[0].imshow(image[:, :, ::-1])
im_handle = ax[1].imshow(saliency_map, cmap='inferno')
plt.colorbar(im_handle)

# Computing saliency of regions in movie frames

## List available sessions of VEDB data

In [ ]:
sessions = sorted(glob.glob('/home/data/vedb/*'))
sessions

In [ ]:
# Grab one session
ses = Session(folder=sessions[8])

In [ ]:
ses.folder

In [ ]:
# Load frames in at limited resolution (300 x 400 pixels)
# Note that we're loading in BGR color format! 

# Also, we will specify frames rather than time to make this 
# easier to map to gaze
start = int(30 * 60 * 3.8) # 3.2 mins in
fin = start + 30 # 30 frames later
world_time, world_video = ses.load('world_camera', 
                                  size=(300,400), 
                                  color='bgr', 
                                  time_idx=None,
                                  frame_idx=(start, fin))

In [ ]:
# 2 relevant dimensions for frame shape are these:
image_shape = world_video.shape[1:3]
frame_number = 0
saliency_generator = pySaliencyMap(*image_shape)
# "SM" here is "Saliency Map" - so, Saliency Map -> get Saliency Map!
saliency_map = saliency_generator.SMGetSM(world_video[frame_number])
# Show map
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
# For display, convert image back to RGB by reversing last axis
_ = ax[0].imshow(world_video[frame_number, :, :, ::-1])
im_handle = ax[1].imshow(saliency_map, cmap='inferno')

Cool! Now that we have saliency, let's have a look at where the participant was looking on this frame. We have gaze data for each session in 

`/home/data/vedb_processed/<folder>/gaze_2D_PL_left_eye.csv`
and 
`/home/data/vedb_processed/<folder>/gaze_2D_PL_right_eye.csv`

In [ ]:
# Import gaze file with pandas
import pandas as pd

In [ ]:
gaze = pd.read_csv('/home/data/vedb_processed/%s/gaze_2D_PL_left_eye.csv'%ses.folder)

In [ ]:
gaze

Plot gaze on the saliency map!

In [ ]:
vdim, hdim = image_shape

In [ ]:
gaze.norm_pos_y[start]

A note: When plotting gaze (or any quantity) on top of an image, you have to make sure you get the Y coordinate correct. Convention for images is to have zero at the top of the Y axis; so make sure your Y variable is correctly displayed (Sometimes y values follow graph convention instead of image convention, and go UP fromthe bottom)

In [ ]:
# Show map
j = 22
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
# For display, convert image back to RGB by reversing last axis
_ = ax[0].imshow(world_video[frame_number + j, :, :, ::-1])
_ = ax[1].imshow(saliency_map, cmap='inferno')
# Gaze is in green 
ax[1].plot(gaze.norm_pos_x[start+j] * hdim, (gaze.norm_pos_y[start+j]) * vdim, 'go')

Play with these saliency maps a bit more! 

Ideas for exploration: 
* compute saliency for more frames
* compute saliency for other sessions (other videos)
* See if you can think of a way to assess how good the saliency map is at capturing where people look - how would you EVALUATE the model? what would you compute? 


Ideas for final projects:
* How is saliency related to the presence of `<x>`?  where `<x>` can be bodies, objects, or other labeled things?